<a href="https://colab.research.google.com/github/SamarpanDas/Face-Recognition-Face-Verification/blob/master/FaceR%26V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving meOne.jpg to meOne.jpg
Saving maOne.jpg to maOne.jpg


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
#import tensorflow as tf
#import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()


In [ ]:
tf.executing_eagerly()

True

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

#np.set_printoptions(threshold=np.nan)

In [ ]:

from fr_utils import *
from inception_blocks_v2 import *


# 1st Model (Inception V2)

In [ ]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
FRmodel.summary()

Model: "FaceRecoModel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 96, 96)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 3, 102, 102)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 48, 48)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 64, 48, 48)   256         conv1[0][0]                      
______________________________________________________________________________________

In [ ]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


.......


.....


# 2nd Model (Inception V3)

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

--2020-09-15 10:18:15--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   190MB/s    in 0.4s    

2020-09-15 10:18:16 (190 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
InceptionV3 = InceptionV3(input_shape=(3,96,96), include_top=False, weights = None)

InceptionV3.load_weights(local_weights_file)

'''
# Make all the layers in the pre-trained model non-trainable
for layer in InceptionV3.layers:
  # Your Code Here
  layers.trainable = False
  
# Print the model summary
InceptionV3.summary()
'''
InceptionV3.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 3, 96, 96)]  0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 32, 47, 47)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 32, 47, 47)   96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_319 (Activation)     (None, 32, 47, 47)   0           batch_normalization_282[0][0]    
_______________________________________________________________________________________

In [ ]:
last_layer = InceptionV3.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


last layer output shape:  (None, 768, 4, 4)


.....

....

...

....

.......

# TRIPLET LOSS


In [ ]:

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    #print(anchor.shape)
    
   
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0), axis = None)
   
    return loss

loss = 528.1426


.....

....

...

....

....

## **COMPILING THE MODEL**

Only compiling model 1 (Inceptionv2) as of now

In [ ]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [ ]:
# done for the day : 15.09.2020

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


In [ ]:
with  tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))

loss = 528.1426


In [ ]:
tf.executing_eagerly()

True

In [ ]:
database = {}
database["samarpan"] = img_to_encoding("meOne.jpg", FRmodel)
database["kaberi"] = img_to_encoding("maOne.jpg", FRmodel)


In [ ]:
def prepare_database():
    database = {}
    database["samarpan"] = img_to_encoding("meOne.jpg", FRmodel)
    database["kaberi"] = img_to_encoding("maOne.jpg", FRmodel)
    return database

In [ ]:
print(len(database))

In [ ]:
tf.executing_eagerly()

True

.....

.......

.......

.......

.....

# **FACE VERIFICATION METHOD**

In [ ]:
def faceVerification(image_path, key, database, model):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
    image_path -- path to an image
    identity -- string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
    database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
    model -- your Inception model instance in Keras
    
    Returns:
    dist -- distance between the image_path and the image of "identity" in the database.
    door_open -- True, if the door should open. False otherwise.
    """
    
 
    
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding - database[key])
    
    # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
    if dist < 0.7:
        print("It's " + str(key) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(key) + ", please go away")
        door_open = False
        
  
        
    return dist, door_open

.....

......

......

# Face Verification Operation

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
faceVerification("maTester.jpg", "maa", database, FRmodel)

.......

.......

.......

........

# **FACE RECOGNITION METHOD**